<a href="https://colab.research.google.com/github/niteshctrl/face_recognition/blob/main/53_Face_Recognition_using_Keras_and_OpenCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing Libraries

import os
import cv2
import glob
import numpy as np

from matplotlib import pyplot
from matplotlib.patches import Rectangle

from keras import Model
from keras.applications import VGG16
from keras.layers import Flatten, Dense
from keras.callbacks import ModelCheckpoint
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
!pwd

/content


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip /content/gdrive/MyDrive/face_data.zip

Archive:  /content/gdrive/MyDrive/face_data.zip
   creating: face_data/
   creating: face_data/train/
   creating: face_data/train/nita/
  inflating: face_data/train/nita/19.jpg  
  inflating: face_data/train/nita/20.jpg  
  inflating: face_data/train/nita/21.jpg  
  inflating: face_data/train/nita/22.jpg  
  inflating: face_data/train/nita/23.jpg  
  inflating: face_data/train/nita/24.jpg  
  inflating: face_data/train/nita/25.jpg  
  inflating: face_data/train/nita/26.jpg  
  inflating: face_data/train/nita/27.jpg  
  inflating: face_data/train/nita/28.jpg  
  inflating: face_data/train/nita/29.jpg  
  inflating: face_data/train/nita/30.jpg  
  inflating: face_data/train/nita/31.jpg  
  inflating: face_data/train/nita/32.jpg  
  inflating: face_data/train/nita/33.jpg  
  inflating: face_data/train/nita/34.jpg  
  inflating: face_data/train/nita/35.jpg  
  inflating: face_data/train/nita/36.jpg  
  inflating: face_data/train/nita/37.jpg  
  inflating: face_data/train/nita/38.jpg  
  i

In [ ]:
# Face detector model

face_detector = cv2.CascadeClassifier('haarcascade.xml')

data_dir = 'face_data/'
image_size = [224,224]

In [ ]:
faces = face_detector.detectMultiScale(cv2.imread('file.jpg'), 1.1, 4)

# Data Collection

In [ ]:
print("Enter the name of the person to collect face data")
name = input()
pwd = !pwd


# Creating directory for storing collected data

if not os.path.isdir(data_dir + name):
    os.mkdir(data_dir + name)


# Initialize the webcam
vid = cv2.VideoCapture(0)
count = 200 # How many frames to collect for training
c = 0


while True:
    ret, frame = vid.read()
    faces = face_detector.detectMultiScale(frame, 1.1, 4)
    
    if len(faces) == 0:
        continue
    c = c + 1
    if c > 200:
        break
    
    for (x, y, w, h) in faces:
        cropped_face = cv2.resize(frame[y:y+h+10, x:x+w+10], image_size)
    cv2.imshow('Video', cropped_face)
    cv2.imwrite(data_dir + name + '/' + str(c) + '.jpg', cropped_face)
    
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

vid.release()
cv2.destroyAllWindows()

Enter the name of the person to collect face data
nita


# Modelling and Classification

In [ ]:
num_classes = glob.glob('face_data/train/*')

In [ ]:
num_classes

['face_data/train/nita', 'face_data/train/nitesh']

In [ ]:
model = VGG16(include_top=False, 
              weights='imagenet',
              input_shape=image_size + [3])

58892288/58889256 [==============================] - 0s 0us/step


In [ ]:
#Freeze the pretrained layers

for layer in model.layers:
    layer.trainable = False

In [ ]:
# Append the fully connected layers

flatten_layer = Flatten()(model.output)
dense1 = Dense(128, activation='relu')(flatten_layer)
final_layer = Dense(len(num_classes), activation='sigmoid')(dense1)

In [ ]:
# Create final model object

my_model = Model(inputs=model.input, outputs=final_layer)
my_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
# Image Augmentation

datagen_train = ImageDataGenerator(rotation_range=20,  # randomly rotate images in the range (degrees, 0 to 180)
                            zoom_range = 0.1, # Randomly zoom image 
                            horizontal_flip=True,  # randomly flip images
                            brightness_range=[0.3,2],
                            rescale = 1./255,
                            validation_split=0)
datagen_val = ImageDataGenerator(rescale=1./255)


iterator_train = datagen_train.flow_from_directory(data_dir+'/train', 
                                       target_size=(224,224),
                                       batch_size=32,
                                       class_mode='binary')

iterator_val = datagen_val.flow_from_directory(data_dir+'/val', 
                                               target_size=(224,224),
                                               batch_size=32,
                                               class_mode='binary')

Found 324 images belonging to 2 classes.
Found 76 images belonging to 2 classes.


In [ ]:
# Compile the model

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy']
)

In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
# Fit the model and save losses to history variable

filepath="my_model.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=True, mode='max')
callbacks_list = [checkpoint]

history = model.fit(iterator_train,
                    validation_data=iterator_val,
                    epochs=10)

Epoch 1/10


InvalidArgumentError: ignored

# References

1. [Face Detection in 2 Minutes using OpenCV & Python] https://towardsdatascience.com/face-detection-in-2-minutes-using-opencv-python-90f89d7c0f81

2. [Python OpenCV: Capture Video from Camera] https://www.geeksforgeeks.org/python-opencv-capture-video-from-camera/

3. [TensorFlow Core v2.5.0] https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit